Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [14]:
NAME = "Damir"
COLLABORATORS = ""

---

# Rosetta Energy Score Functions

A basic function of Rosetta is calculating the energy or score of a biomolecule. Rosetta has a standard energy function for all-atom calculations as well as several scoring functions for low-resolution protein representations. See https://www.ncbi.nlm.nih.gov/pubmed/28430426 for a review on the all-atom score functions.

You can also tailor an energy function by including scoring terms of your choice with custom weights.

**Chapter contributors:**

- Kathy Le (Johns Hopkins University) adapted this chapter from the [PyRosetta Workshops](https://www.amazon.com/PyRosetta-Interactive-Platform-Structure-Prediction-ebook/dp/B01N21DRY8) (J. J. Gray, S. Chaudhury, S. Lyskov, J. Labonte, 2017).

# Score Function Basics
Keywords: score function, ScoreFunction(), get_score_function(), set_weight(), show(), etable_atom_pair_energies(), Atom objects, get_hbonds(), nhbonds(), residue_hbonds()

In [15]:
# This takes ~1.5 minutes each time you start a new notebook or do a "factory reset".
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.mount_pyrosetta_install() #Instead of pyrosettacolabsetup.setup
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")


Drive already mounted at /content/google_drive; to attempt to forcibly remount, call drive.mount("/content/google_drive", force_remount=True).
Notebook is set for PyRosetta use in Colab.  Have fun!


In [16]:
import pyrosetta
pyrosetta.init()

PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2021.21+release.882e5c1ab85c8c251fce4eb3e1e0504af590786a 2021-05-26T14:40:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python37.ubuntu r284 2021.21+release.882e5c1ab85 882e5c1ab85c8c251fce4eb3e1e0504af590786a http://www.pyrosetta.org 2021-05-26T14:40:53
core.init: command: PyRosetta -ex1 -ex2aro -database /content/google_drive/MyDrive/prefix/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1913879660 seed_offset=0 real_seed=1913879660
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=1913879660 RG_type=mt19937


**Make sure you are in the directory with the pdb files:**



In [17]:
%cd google_drive/My\ Drive/temp_pyrbc_202103_notebooks/

[Errno 2] No such file or directory: 'google_drive/My Drive/temp_pyrbc_202103_notebooks/'
/content/google_drive/My Drive/temp_pyrbc_202103_notebooks


In this module, we will explore the PyRosetta score function interface. You will learn to inspect energies of a biomolecule at the whole protein, per-residue, and per-atom level. Finally, you will gain practice applying the energies to answering biological questions involving proteins. For these exercises, we will use the protein Ras (PDB 6q21). You should have the PDB file "6Q21_A.pdb" in your `inputs/` directory. Load it into a Pose variable named `ras` with the pyrosetta.pose_from_pdb method. 

In [18]:
ras = pyrosetta.pose_from_pdb("inputs/6Q21_A.pdb")
print(ras)

core.import_pose.import_pose: File 'inputs/6Q21_A.pdb' automatically determined to be of type PDB
PDB file name: inputs/6Q21_A.pdb
Total residues: 171
Sequence: MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVIDGETCLLDILDTAGQEEYSAMRDQYMRTGEGFLCVFAINNTKSFEDIHQYREQIKRVKDSDDVPMVLVGNKCDLAARTVESRQAQDLARSYGIPYIETSAKTRQGVEDAFYTLVREIRQHKLRKL
Fold tree:
FOLD_TREE  EDGE 1 171 -1 


To score a protein, you will begin by defining a score function using the `get_score_function(is_fullatom: bool)` method in the `pyrosetta.teaching` namespace. Specifying `True` will return the default `ref2015` all-atom energy function, while `False` will specify the default centroid score function.

Create a PyRosetta score function using:
```
sfxn = get_score_function(True)
```

In [19]:
from pyrosetta.teaching import *

sfxn = get_score_function(True)

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015


You can see the terms, weights, and energy method options by printing the score function:

```
print(sfxn)
```

In [20]:
print(sfxn)

ScoreFunction::show():
weights: (fa_atr 1) (fa_rep 0.55) (fa_sol 1) (fa_intra_rep 0.005) (fa_intra_sol_xover4 1) (lk_ball_wtd 1) (fa_elec 1) (pro_close 1.25) (hbond_sr_bb 1) (hbond_lr_bb 1) (hbond_bb_sc 1) (hbond_sc 1) (dslf_fa13 1.25) (omega 0.4) (fa_dun 0.7) (p_aa_pp 0.6) (yhh_planarity 0.625) (ref 1) (rama_prepro 0.45)
energy_method_options: EnergyMethodOptions::show: aa_composition_setup_files: 
EnergyMethodOptions::show: mhc_epitope_setup_files: 
EnergyMethodOptions::show: netcharge_setup_files: 
EnergyMethodOptions::show: aspartimide_penalty_value: 25
EnergyMethodOptions::show: etable_type: FA_STANDARD_DEFAULT
analytic_etable_evaluation: 1
EnergyMethodOptions::show: method_weights: ref 1.32468 3.25479 -2.14574 -2.72453 1.21829 0.79816 -0.30065 2.30374 -0.71458 1.66147 1.65735 -1.34026 -1.64321 -1.45095 -0.09474 -0.28969 1.15175 2.64269 2.26099 0.58223
EnergyMethodOptions::show: method_weights: free_res
EnergyMethodOptions::show: unfolded_energies_type: UNFOLDED_SCORE12
EnergyMeth

## Practice: List the terms in the energy function and their relative weights

**Hint:** look at the top line that starts with 'weights'

In [21]:
print(sfxn.weights())

( fa_atr; 1) ( fa_rep; 0.55) ( fa_sol; 1) ( fa_intra_atr; 0) ( fa_intra_rep; 0.005) ( fa_intra_sol; 0) ( fa_intra_atr_xover4; 0) ( fa_intra_rep_xover4; 0) ( fa_intra_sol_xover4; 1) ( fa_intra_atr_nonprotein; 0) ( fa_intra_rep_nonprotein; 0) ( fa_intra_sol_nonprotein; 0) ( fa_intra_RNA_base_phos_atr; 0) ( fa_intra_RNA_base_phos_rep; 0) ( fa_intra_RNA_base_phos_sol; 0) ( fa_atr_dummy; 0) ( fa_rep_dummy; 0) ( fa_sol_dummy; 0) ( fa_vdw_tinker; 0) ( lk_hack; 0) ( lk_ball; 0) ( lk_ball_wtd; 1) ( lk_ball_iso; 0) ( lk_ball_bridge; 0) ( lk_ball_bridge_uncpl; 0) ( coarse_fa_atr; 0) ( coarse_fa_rep; 0) ( coarse_fa_sol; 0) ( coarse_beadlj; 0) ( mm_lj_intra_rep; 0) ( mm_lj_intra_atr; 0) ( mm_lj_inter_rep; 0) ( mm_lj_inter_atr; 0) ( mm_twist; 0) ( mm_bend; 0) ( mm_stretch; 0) ( lk_costheta; 0) ( lk_polar; 0) ( lk_nonpolar; 0) ( lk_polar_intra_RNA; 0) ( lk_nonpolar_intra_RNA; 0) ( fa_elec; 1) ( fa_elec_bb_bb; 0) ( fa_elec_bb_sc; 0) ( fa_elec_sc_sc; 0) ( fa_intra_elec; 0) ( h2o_hbond; 0) ( dna_dr; 0) 

## Custom energy functions

You can also create a custom energy function that includes select terms. Typically, creating a whole new score function is unneccesary because the current one works well in most cases. However, tweaking the current enrgy function by reassigning weights and adding certain energy terms can be useful.

Here, we will make an example energy function with only the van der Waals attractive and repulsive terms, both with weights of 1. We need to use the `set_weight()`. Make a new `ScoreFunction` and set the weights accordingly. This is how we set the full-atom attractive (`fa_atr`) and the full-atom repulsive (`fa_rep`) terms.

```
sfxn2 = ScoreFunction()
sfxn2.set_weight(fa_atr, 1.0)
sfxn2.set_weight(fa_rep, 1.0)
```

In [22]:
sfxn2 = ScoreFunction()
sfxn2.set_weight(fa_atr, 1.0)
sfxn2.set_weight(fa_rep, 1.0)

print(sfxn2.weights())

( fa_atr; 1) ( fa_rep; 1) ( fa_sol; 0) ( fa_intra_atr; 0) ( fa_intra_rep; 0) ( fa_intra_sol; 0) ( fa_intra_atr_xover4; 0) ( fa_intra_rep_xover4; 0) ( fa_intra_sol_xover4; 0) ( fa_intra_atr_nonprotein; 0) ( fa_intra_rep_nonprotein; 0) ( fa_intra_sol_nonprotein; 0) ( fa_intra_RNA_base_phos_atr; 0) ( fa_intra_RNA_base_phos_rep; 0) ( fa_intra_RNA_base_phos_sol; 0) ( fa_atr_dummy; 0) ( fa_rep_dummy; 0) ( fa_sol_dummy; 0) ( fa_vdw_tinker; 0) ( lk_hack; 0) ( lk_ball; 0) ( lk_ball_wtd; 0) ( lk_ball_iso; 0) ( lk_ball_bridge; 0) ( lk_ball_bridge_uncpl; 0) ( coarse_fa_atr; 0) ( coarse_fa_rep; 0) ( coarse_fa_sol; 0) ( coarse_beadlj; 0) ( mm_lj_intra_rep; 0) ( mm_lj_intra_atr; 0) ( mm_lj_inter_rep; 0) ( mm_lj_inter_atr; 0) ( mm_twist; 0) ( mm_bend; 0) ( mm_stretch; 0) ( lk_costheta; 0) ( lk_polar; 0) ( lk_nonpolar; 0) ( lk_polar_intra_RNA; 0) ( lk_nonpolar_intra_RNA; 0) ( fa_elec; 0) ( fa_elec_bb_bb; 0) ( fa_elec_bb_sc; 0) ( fa_elec_sc_sc; 0) ( fa_intra_elec; 0) ( h2o_hbond; 0) ( dna_dr; 0) ( dna_b

Lets compare the score of `ras` using the full-atom `ScoreFunction` versus the `ScoreFunction` we made above using only the attractive and repulsive terms.

First, print the total energy of `ras` using `print(sfxn(ras))`
Then, print the attractive and repulsive energy only of `ras` using `print(sfxn2(ras))`

In [23]:
# print total energy of ras
print(sfxn(ras))

# print the attractive and repulsive energy of ras
print(sfxn2(ras))

1215.729070042399
154.59159174026854


## Energy Breakdown

Using the full-atom `ScoreFunction` `sfxn`, break the energy of `ras` down into its individual pieces with the `sfxn.show(ras)` method. Which are the three most dominant contributions, and what are their values? Is this what you would have expected? Why? Note which terms are positive and negative

In [24]:
# use the sfxn.show() method
sfxn.show(ras)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000   -1039.246   -1039.246
 fa_rep                       0.550    1193.837     656.611
 fa_sol                       1.000     682.582     682.582
 fa_intra_rep                 0.005     700.419       3.502
 fa_intra_sol_xover4          1.000      46.564      46.564
 lk_ball_wtd                  1.000     -14.597     -14.597
 fa_elec                      1.000    -195.387    -195.387
 pro_close                    1.250      97.210     121.513
 hbond_sr_bb                  1.000     -41.656     -41.656
 hbond_lr_bb                  1.000     -28.352     -28.352
 hbond_bb_sc                  1.000     -13.111     -13.111
 hbond_sc                     1.000      -7.771      -7.771
 dslf_fa13                    1.250       0.000       0.000
 omega  

The three most dominant contributions are fa_atr (-1039.246), fa_dun (-907.650), and fa_sol (682.582)

Unweighted, individual component energies of each residue in a structure are stored in the `Pose` object and can be accessed by the `energies()` method. For example, to break down the energy into each residue's contribution, we use: 
```
print(ras.energies().show(<n>))
```
Where `<n>` is the residue number.

What are the total van der Waals, solvation, and hydrogen-bonding contributions of residue 24?

Note: The _backbone_ hydrogen-bonding terms for each residue are not available from the `Energies` object. You can get them by using EnergyMethodOptions. See http://www.pyrosetta.org/documentation#TOC-Hydrogen-Bonds-and-Hydrogen-Bond-Scoring.

In [25]:
print(ras.energies().show(24))

core.scoring.Energies: E               fa_atr        fa_rep        fa_sol  fa_intra_repfa_intra_sol_x   lk_ball_wtd       fa_elec     pro_close   hbond_sr_bb   hbond_lr_bb   hbond_bb_sc      hbond_sc     dslf_fa13         omega        fa_dun       p_aa_pp yhh_planarity           ref   rama_prepro
core.scoring.Energies: E(i)  24         -7.40         19.03          2.94          8.76          0.09         -0.11         -0.56          0.00          0.00          0.00          0.00          0.00          0.00          0.12          2.68          0.06          0.00          2.30          3.58
None


The van der Waals, solvation, and electrostatic terms are atom-atom pairwise energies calculated from a pre-tabulated lookup table, dependent upon the distance between the two atoms and their types. You can access this lookup table, called the `etable` directly to check these energy calculations on an atom-by-atom basis. Use the `etable_atom_pair_energies` function which returns a triplet of energies for attractive, repulsive and solvation scores.

(Note that the `etable_atom_pair_energies()` function requires `Atom` objects, not the `AtomID` objects we saw in Workshop #2. For more info, look at the [documentation](https://graylab.jhu.edu/PyRosetta.documentation/pyrosetta.toolbox.atom_pair_energy.html?highlight=etable_atom_pair_energies#pyrosetta.toolbox.atom_pair_energy.etable_atom_pair_energies).)

**Practice:** What are the attractive, repulsive, solvation, and electrostatic components between the nitrogen of residue 24 and the oxygen of residue 20? 


In [26]:
res_24 = ras.residue(24)
res_20 = ras.residue(20)
nitrogen_index_24 = res_24.atom_index('N')
oxygen_index_20 = res_20.atom_index('O')

solv = pyrosetta.etable_atom_pair_energies(res_24, nitrogen_index_24, res_20, oxygen_index_20, sfxn)
solv

(-0.1505855046001568, 0.0, 0.5903452111877214, 2.173111777247698)

# Practice: Analyzing energy between residues
Keywords: pose_from_rcsb(), pdb2pose(), EMapVector()

In [27]:
# From previous section:
sfxn = get_score_function(True)

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015


Analyze the energy between residues Y102 and Q408 in cetuximab (PDB code 1YY9, use the `pyrosetta.toolbox.pose_from_rcsb` function to download it and load it into a new `Pose` object) by following the steps below. 

A. Internally, a Pose object has a list of residues, numbered starting from 1. To find the residue numbers of Y102 of chain D and Q408 of chain A, use the residue chain identifier and the PDB residue number to convert to the pose numbering using the `pose2pdb()` method:

```
pose = pyrosetta.toolbox.pose_from_rcsb("1YY9")
res102 = pose.pdb_info().pdb2pose("D", 102)
res408 = pose.pdb_info().pdb2pose("A", 408)
```

In [29]:
pose = pyrosetta.toolbox.pose_from_rcsb("1YY9")
res102 = pose.pdb_info().pdb2pose("D", 102)
res408 = pose.pdb_info().pdb2pose("A", 408)

core.import_pose.import_pose: File '1YY9.clean.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue GLU:NtermProteinFull 1
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue GLU:NtermProteinFull 1
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE1 on residue GLU:NtermProteinFull 1
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE2 on residue GLU:NtermProteinFull 1
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue GLU 2
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue GLU 2
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE1 on residue GLU 2
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE2 on residue GLU 2
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue LYS 3
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on r

B. Score the pose and determine the van der Waals energies and solvation energy between these two residues. Use the following commands to isolate contributions from particular pairs of residues, where `rsd102` and `rsd408` are the two residue objects of interest from above (not the residue number -- use `pose.residue(res_num)` to access the objects): 

```
emap = EMapVector()
sfxn.eval_ci_2b(pose.residue(res102), pose.residue(res408), pose, emap)
print(emap[fa_atr])
print(emap[fa_rep])
print(emap[fa_sol])
```

In [30]:
emap = EMapVector()
sfxn.eval_ci_2b(pose.residue(res102), pose.residue(res408), pose, emap)
print(emap[fa_atr])
print(emap[fa_rep])
print(emap[fa_sol])

-1.2098840439684349
0.10835353848860374
1.5729435146961963


# Energies and the PyMOL Mover
Keywords: send_energy(), label_energy(), send_hbonds()

In [31]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.mount_pyrosetta_install() #Instead of pyrosettacolabsetup.setup
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

from pyrosetta import *
from pyrosetta.teaching import *
init()

Drive already mounted at /content/google_drive; to attempt to forcibly remount, call drive.mount("/content/google_drive", force_remount=True).
Notebook is set for PyRosetta use in Colab.  Have fun!
PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2021.21+release.882e5c1ab85c8c251fce4eb3e1e0504af590786a 2021-05-26T14:40:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python37.ubuntu r284 2021.21+release.882e5c1ab85 882e5c1ab85c8c251fce4eb3e1e0504af590786a http://www.pyrosetta.org 2021-05-26T14:40:53
core.init: command: PyRosetta -ex1 -ex2aro -database /content/google_drive/MyDrive/prefix/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-1761385165 seed_offset=0 real_seed=-17613

**Make sure you are in the directory that holds your `inputs/` directory:**

`cd google_drive/My\ Drive/pyrbc_202106_notebooks/`

In [32]:
# From previous section:
ras = pyrosetta.pose_from_pdb("inputs/6Q21_A.pdb")
sfxn = get_fa_scorefxn()

core.import_pose.import_pose: File 'inputs/6Q21_A.pdb' automatically determined to be of type PDB
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015


The `PyMOLMover` class contains a method for sending score function information to PyMOL,
which will then color the structure based on relative residue energies.

Open up PyMOL on your computer (and, if you have not added the execution of the `PyMOL-Rosetta-relay-client.python3.py` to your `.pymolrc`, then run it manually). Initialize the commuication inside colab with the relay server at Hopkins. Instantiate a `PyMOLMover` object and use the `pymol_mover.send_energy(ras)` to send the coloring  command to PyMOL.

```
if os.getenv("DEBUG"): sys.exit(0)
import pyrosetta.network

# make sure to supply unique secret here and use _the same_ secrete when running PyMOL-Rosetta-relay-client.python3.py in PyMOL on your desktop machine
pyrosetta.network.start_udp_to_tcp_bridge_daemon(secret='my-super-unique-secret-please-change-me')


pymol_mover = PyMOLMover()
pymol_mover.apply(ras)
print(sfxn(ras))
pymol_mover.send_energy(ras)
```

In [33]:
if os.getenv("DEBUG"): sys.exit(0)
import pyrosetta.network
 
# make sure to supply unique secret here and use _the same_ secrete when running PyMOL-Rosetta-relay-client.python3.py in PyMOL on your desktop machine
pyrosetta.network.start_udp_to_tcp_bridge_daemon(secret='none')
 
 
pymol_mover = PyMOLMover()
pymol_mover.apply(ras)
print(sfxn(ras))
pymol_mover.send_energy(ras)

UDP server started: localhost:65000...
1215.729070042399


In [34]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython import display

In [35]:
from pathlib import Path
gifPath = Path("./Media/PyMOL-send_energy.gif")
# Display GIF in Jupyter, CoLab, IPython
with open(gifPath,'rb') as f:
    display.Image(data=f.read(), format='png',width='800')

What color is residue Proline34? What color is residue Alanine66? Which residue has lower energy?

In [ ]:
# your response here

`pymol_mover.send_energy(ras, fa_atr)` will have PyMOL color only by the attractive van der Waals energy component. What color is residue 34 if colored by solvation energy, `fa_sol`?

In [ ]:
# send specific energies to pymol

In [36]:
pymol_mover.send_energy(ras, fa_atr)

You can have PyMOL label each Cα with the value of its residue’s specified energy using:
```
pymol_mover.label_energy(ras, "fa_atr")
```

In [37]:
pymol_mover.label_energy(ras, "fa_atr")

Finally, if you have scored the `pose` first, you can have PyMOL display all of the calculated hydrogen bonds for the structure:

```
pymol_mover.send_hbonds(ras)
```

In [38]:
pymol_mover.send_hbonds(ras)

## References
This Jupyter notebook is an adapted version of "Workshop #3: Scoring" in the PyRosetta workbook: https://graylab.jhu.edu/pyrosetta/downloads/documentation/pyrosetta4_online_format/PyRosetta4_Workshop3_Scoring.pdf